In [1]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
import os
os.chdir('/Users/kathleenhamilton/Desktop/QHACK2021/qose/QHACK2021/src')


## Import data from sklearn

In [36]:
from sklearn import datasets

In [37]:
n_samples = 100
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)

In [38]:
X_train = noisy_circles[0]
Y_train = noisy_circles[1]
X_test = noisy_circles[0][50:]
Y_test = noisy_circles[1][50:]

### Try running a loop over some hyper_parameters

This uses the circuit classifer built during the Challenge -- the circuit and QNode is constructed inside the function `classify_data`

The following characteristics are hard wired inside the function `classify_data`:
* number of qubits (3)
* number of rotation gates (6)
* the initialization used for the rotations and weights (rotations intialized with 0, weights initialized with random numbers drawn uniformly from $[-2.5,2.5]$
* the rotation gates that are trained (RY)
* the rotation gates used in the `AngleEmbeddding` (RY)
* the optimizer (`AdamOptimizer`)

The following parameters are passed as keywords:
* `s` (the number of steps to train for)
* `batch_size` (the batch size used in training)
* `learning_rate` (the initial learning rate for Adam)

As in (de Wynter 2020) we only train each circuit for a few steps (here I'm using 10).  In (de Wynter 2020) the error rate surrogate is defined using a loss function evaluated over a subset of the data -- here I'm using the accuracy of assigned labels over the test data.  The full number of samples that I generated for the dataset is given by `n_samples` (above).  I've split that data in to train,test sets. 

In [39]:
batch_sets = [2,4,8]
learning_rates = [0.01,0.05]
hyperparameters = list(itertools.product(batch_sets,learning_rates))
print(hyperparameters)
for idx,sdx in hyperparameters:
    print(idx,sdx,classify_data(X_train,Y_train,X_test,Y_test,s=3,batch_size=idx,learning_rate=sdx))

[(2, 0.01), (2, 0.05), (4, 0.01), (4, 0.05), (8, 0.01), (8, 0.05)]


NameError: name 'classify_data' is not defined

In [40]:
#uncommenting and running this cell should cause an error
#inside_circuit 

In [ ]:
X_train.shape[1]

### Try running a loop over some hyper_parameters

This time use a circuit (QNode) that is created outside the function and passed as an argument

Things that are still hard-wired inside the `train_circuit` function:
* Optimizer choice
* Initialization choice (same as above)


In [41]:
def variational_circuit(params,features=None):
    r_ = QUBITS//len(features)
    large_features = np.tile(features,r_)
    qml.templates.embeddings.AngleEmbedding(large_features, wires=range(QUBITS), rotation='Y') # replace with more general embedding
    if len(params)%QUBITS!=0:
        print('chooose parameter length that is divisible by number of qubits')
        return
    W= np.reshape(params,(len(params)//QUBITS,QUBITS))
    qml.templates.layers.BasicEntanglerLayers(W, wires=range(QUBITS), rotation=qml.ops.RY)
    return [qml.expval(qml.PauliZ(idx)) for idx in range(QUBITS)]


In [42]:
qubit_sizes=[2,4]
batch_sets = [2,4,8,16,32]
learning_rates = [0.001,0.005,0.01,0.05]
hyperparameters = list(itertools.product(batch_sets,learning_rates))
print(hyperparameters)
results = {}
Wmax = 0.0
for qdx in qubit_sizes:
    QUBITS=qdx
    dev = qml.device("default.qubit",wires=QUBITS)
    # Instantiate the QNode
    outside_circuit = qml.QNode(func=variational_circuit,device=dev)
    n_params=2*QUBITS
    for idx,sdx in hyperparameters:
        p,i,er,wtemp,weights=train_circuit(outside_circuit,n_params,qdx,X_train,Y_train,s=5,batch_size=idx,rate_type='accuracy',learning_rate=sdx)
        print(p,i,er,wtemp,weights)
        if wtemp>=Wmax:
            Wmax=wtemp
            saved_weights = weights
            output = (idx,sdx,p,i,er)
print("Max W coef: ", Wmax)
print("saved weights: ",saved_weights)
print("hyperparameters: ",output[0],output[1])
print("variables (p,i,er): ",p,i,er)

[(2, 0.001), (2, 0.005), (2, 0.01), (2, 0.05), (4, 0.001), (4, 0.005), (4, 0.01), (4, 0.05), (8, 0.001), (8, 0.005), (8, 0.01), (8, 0.05), (16, 0.001), (16, 0.005), (16, 0.01), (16, 0.05), (32, 0.001), (32, 0.005), (32, 0.01), (32, 0.05)]
0 0.8152356062630997
1 0.7781295080909417
2 0.024832276013561454
3 0.0002476621335848022
4 0.3795616755602387
6 0.0035056114196777345 0.400001 2.349911743352279 [ 0.00149789 -0.00271719  0.00375639 -0.0026978   1.08712593 -0.75335642]
0 0.021033007102673446
1 0.617130502953682
2 0.023121963410116957
3 0.605890365469401
4 0.312241545343045
6 0.0034896135330200195 0.600001 1.5666093850390075 [-0.00577472 -0.00805874  0.00558401 -0.00736023  1.4524959   1.24238114]
0 0.22930810052470318
1 0.25606225399830634
2 0.18624364555634954
3 1.3131123763047956
4 1.2428629921916117
6 0.003441929817199707 0.8000010000000001 1.174958088627037 [ 0.03203972 -0.03027632 -0.03531866 -0.02995925 -0.04395668 -1.28154022]
0 0.5182955333508472
1 0.32395975310065295
2 0.03054

0 2.0368054320238844
1 2.7985338627702077
2 0.8614466124991804
3 2.01269317664536
4 1.2482635193776985
12 0.009786975383758546 1.000001 0.8799129947036282 [ 0.03067722 -0.02883939  0.02923708 -0.02897113  0.04245457  0.03773675
  0.03880767  0.02849932 -1.39335604 -1.32315808 -1.8980504  -1.56780207]
0 2.0388860770700297
1 1.9165722038636235
2 1.8317303696315586
3 2.201507109270847
4 2.3517480518285723
12 0.009139525890350341 0.575001 1.530292246747684 [ 0.20378327  0.16262763 -0.12939689  0.16549718  0.16740488  0.16857379
  0.22410773  0.19220106 -2.08982941  1.03919083 -1.62594355  0.16178249]
0 1.0820092031164783
1 0.9426052925876078
2 1.2373709866113076
3 1.4027151449521469
4 1.4306062271341355
12 0.00987546443939209 0.587501 1.4977218692613856 [ 0.00335911  0.00292679  0.00336066  0.00261531  0.00463341  0.0043493
 -0.00394358 -0.00300236  0.19502732 -1.54997315  0.77304918 -0.54320854]
0 0.5043869053710804
1 0.49353058585481
2 0.6243340987798196
3 0.9556238660653114
4 0.76225994

In [ ]:
loop_over_hyperparameters(outside_circuit,4,X_train,Y_train,batch_sets=[2,4,8,16,32],learning_rates=[0.001,0.005,0.01,0.05],s=5,rate_type='accuracy')

### Pull in more detailed circuit design

In [ ]:
import sys
import pennylane as qml
import sklearn as skl
import autograd.numpy as np
import itertools
import time

In [15]:


def zz_layer(wires, params):
    nq = len(wires)
    for n in range(nq - 1):
        zz_gate([n, n + 1], params[n])
    zz_gate([nq - 1, 0], params[nq - 1])


def zz_gate(wires, gamma):
    if isinstance(gamma,(float,qml.variable.Variable)):
        qml.CNOT(wires=wires)
        qml.RZ(gamma, wires=wires[1])
        qml.CNOT(wires=wires)
    else:
        qml.CNOT(wires=wires)
        qml.RZ(gamma._value, wires=wires[1])
        qml.CNOT(wires=wires)

def CNOT_layer(wires):
    nq = len(wires)
    for n in range(nq-1):
        qml.CNOT(wires=[n,n+1])
    qml.CNOT(wires=[nq-1,0])

def x_layer(wires, params):
    nqubits = len(wires)
    if isinstance(params,(list,np.ndarray,qml.variable.Variable)):
        for n in range(nqubits):
            qml.RX(params[n], wires=[n, ])
    else:
        for n in range(nqubits):
            qml.RX(params[n]._value, wires=[n, ])


def y_layer(wires, params):
    nqubits = len(wires)
    if isinstance(params,(list,np.ndarray,qml.variable.Variable)):
        for n in range(nqubits):
            qml.RY(params[n], wires=[n, ])
    else:
        for n in range(nqubits):
            qml.RY(params[n]._value, wires=[n, ])

#TODO: ADD W-COST HERE

string_to_layer_mapping = {'ZZ': zz_layer, 'X': x_layer, 'Y': y_layer}

In [22]:

def string2circuit(gate_params=[],features=[],arch_params=[],n_wires=None):
    string_to_param_count = {'E1':0,'ZZ':n_wires,'X':n_wires,'Y':n_wires}
    pdx = 0
    for gdx in arch_params:
        if gdx=='E1':
            r_ = n_wires//len(features)
            large_features = np.tile(features,r_)
            qml.templates.embeddings.AngleEmbedding(large_features, wires=range(n_wires), rotation='Y') # replace with more general embedding
            qml.templates.embeddings.AngleEmbedding(features, wires=range(n_wires), rotation='Y') # replace with more general embedding
        elif gdx=='ZZ':
            #w = gate_params[pdx:pdx+n_wires]
            #zz_layer(range(n_wires),w)
            CNOT_layer(range(n_wires))
            #pdx+=n_wires
        elif gdx=='X':
            w = gate_params[pdx:pdx+n_wires]
            x_layer(range(n_wires),w)
            pdx+=n_wires
        elif gdx=='Y':
            w = gate_params[pdx:pdx+n_wires]
            y_layer(range(n_wires),w)
            pdx+=n_wires
    return [qml.expval(qml.PauliZ(idx)) for idx in range(n_wires)]
    

In [29]:
batch_sets = [2,4]
learning_rates = [0.001,0.005]
hyperparameters = list(itertools.product(batch_sets,learning_rates))
print(hyperparameters)
results = {}
Wmax = 0.0

ARCH = ['E1','ZZ','Y','ZZ','Y','ZZ','Y','ZZ','Y','ZZ','Y']
WIRES = 2
def temp_circuit(w,features=None): return string2circuit(gate_params=w,features=features,arch_params=ARCH,n_wires=WIRES)

 
dev = qml.device("default.qubit",wires=WIRES)
outside_circuit = qml.QNode(temp_circuit,device=dev)

#if using zz-layer, use this line
#n_params=np.sum([WIRES for x in ARCH if (x!='E1')])
#if useing CNOT-layer, use this line
n_params=np.sum([WIRES for x in ARCH if (x=='X') or (x=='Y')])
print(n_params)
for idx,sdx in hyperparameters:
    p,i,er,wtemp,weights=train_circuit(outside_circuit,n_params,WIRES,X_train,Y_train,s=5,batch_size=idx,rate_type='batch_cost',learning_rate=sdx)
    print('final weights:',weights)
    print(p,i,er,wtemp,weights)
    if wtemp>=Wmax:
        Wmax=wtemp
        saved_weights = weights
        output = (idx,sdx,p,i,er)
print("Max W coef: ", Wmax)
print("saved weights: ",saved_weights)
print("hyperparameters: ",output[0],output[1])
print("variables (p,i,er): ",p,i,er)

[(2, 0.001), (2, 0.005), (4, 0.001), (4, 0.005)]
10
0 1.2881265208682278
1 0.4637075730509477
2 1.8128674559858333
3 0.009572095751300205
4 1.0951033667563221
final weights: [-0.00322823  0.00401292  0.00281863  0.00291129 -0.00292532  0.00273077
 -0.00278882  0.00401279  0.00281819  0.00291234  1.03801061 -1.80680274]
12 0.6920039653778076 1.0951033667563221 0.7980163257950555 [-0.00322823  0.00401292  0.00281863  0.00291129 -0.00292532  0.00273077
 -0.00278882  0.00401279  0.00281819  0.00291234  1.03801061 -1.80680274]
0 0.08928237619553689
1 0.00219601360235117
2 0.15173087675187005
3 0.10694890579485877
4 0.18957856520787697
final weights: [ 3.63591362e-04  1.45364774e-02  1.42460109e-02 -1.07818974e-02
  1.51591647e-02  1.28531386e-02  1.94084311e-03  1.45425864e-02
  1.42426298e-02 -1.05010776e-02  2.21717721e+00  1.61980443e-01]
12 0.6814248561859131 0.18957856520787697 4.610244097518095 [ 3.63591362e-04  1.45364774e-02  1.42460109e-02 -1.07818974e-02
  1.51591647e-02  1.285313